In [ ]:
# !pip install -q transformers
!pip install -q bitsandbytes
# !pip install -q datasets
# !pip install -q torch
!pip install -q lm-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from torch.optim import AdamW
from transformers import (AutoModelForCausalLM, Gemma3ForConditionalGeneration, AutoProcessor,
                          AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments)
from tqdm import tqdm
from datasets import load_dataset
from torch.nn import functional as F
from torch.utils.data import DataLoader

In [ ]:
import os
from google.colab import userdata

os.environ["HF_TOKEN"] =  userdata.get('HF_TOKEN')
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
teacher_model_name = "meta-llama/Llama-3.2-3B"
student_model_name = "meta-llama/Llama-3.2-1B"

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(student_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|end_of_text|>'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# if torch.cuda.get_device_capability()[0] >= 8:
#     !pip install -qqq flash-attn
#     torch_dtype = torch.bfloat16
#     attn_implementation = "flash_attention_2"
# else:
#     torch_dtype = torch.float16
#     attn_implementation = "eager"

torch_dtype = torch.bfloat16
attn_implementation = "eager"

In [ ]:
# quantization_config = BitsAndBytesConfig(load_in_4bit=True,
#                                          bnb_4bit_use_double_quant=True,
#                                          bnb_4bit_quant_type="nf4",
#                                          bnb_4bit_compute_dtype=torch_dtype,
#                                          llm_int8_enable_fp32_cpu_offload=False)

In [ ]:
# teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_name,
#                                                                quantization_config=quantization_config,
#                                                                device_map="auto",
#                                                                trust_remote_code=True,
#                                                                torch_dtype=torch_dtype,
#                                                                attn_implementation=attn_implementation)

# student_model = AutoModelForCausalLM.from_pretrained(student_model_name,
#                                                      quantization_config=quantization_config,
#                                                      device_map="auto",
#                                                      trust_remote_code=True,
#                                                      attn_implementation=attn_implementation)

In [ ]:
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_name,
                                                     device_map="auto")

student_model = AutoModelForCausalLM.from_pretrained(student_model_name,
                                                     device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset = load_dataset("lavita/medical-qa-datasets", "all-processed", split="train")
original_dataset = dataset
dataset = dataset.select(range(1000))

In [ ]:
dataset.column_names

['instruction', 'input', 'output', '__index_level_0__']

In [ ]:
def format_prompt(example):
    return {
        "prompt": f"Instruction: {example['instruction']}\nQuestion{example['input']}\nAnswer: {example['output']}"
    }

formatted_dataset = dataset.map(format_prompt)

In [ ]:
print(formatted_dataset['prompt'][0])

Instruction: If you are a doctor, please answer the medical questions based on the patient's description.
Questionhi. im a home health aide and i have a client with scoliosis in the back and kidney disease. her feet ankles and calves have been swollen for the past 2 weeks. mostly in her feet. she started a patch for pain in her legs 3 weeks ago. she started swelling up almost a week after she started the patch and the pain doctor cut the dose in half and she is still swollen. she has no blood clots in her legs because one of her doctors checked and they said it might be because of her back. what do you think? im concerned because she has been swollen up for to long and theres only so much i can do being her home health aide. we both want to get to the bottom of this swelling she is having
Answer: hi, thanks for contacting chatbot. swelling in the legs and feet can come from many causes, one of them being general circulation or ineffectiveness of the kidneys to rid the body of excess wa

In [ ]:
def tokenize(samples):
    tokenized = tokenizer(samples["prompt"],
                          padding="max_length",
                          truncation=True,
                          max_length=128,
                          return_tensors="pt")

    input_ids = tokenized["input_ids"]
    labels = input_ids.clone()
    attention_mask = tokenized["attention_mask"]

    return{
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask
    }

In [ ]:
print("Tokenizing dataset .... ")
tokenized_dataset = formatted_dataset.map(tokenize,
                                          batched=True,
                                          batch_size=32,
                                          remove_columns=dataset.column_names,
                                          desc="Processing samples",
                                          load_from_cache_file=False)

Tokenizing dataset .... 


Processing samples:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
dataloader = DataLoader(
    tokenized_dataset,
    batch_size=4,
    shuffle=True
)

### Optimizer

In [ ]:
lr = 1e-5
optimizer = AdamW(student_model.parameters(), lr=lr)

### Training Loop

In [ ]:
num_epochs = 10
temperature = 2.0
alpha = 1
accumulation_steps = 8

In [ ]:
# batch = next(iter(dataloader))
# batch

In [ ]:
# teacher_model.eval()
# with torch.no_grad():
#     teacher_outputs = teacher_model(
#         input_ids=batch["teacher_input_ids"].to(device),
#         attention_mask=batch["teacher_attention_mask"].to(device),
#         labels=batch["teacher_labels"].to(device),
#         output_hidden_states=True
#     )
# teacher_outputs.logits

In [ ]:
# student_model.eval()
# with torch.no_grad():
#     student_outputs = student_model(
#         input_ids=batch["student_input_ids"].to(device),
#         attention_mask=batch["student_attention_mask"].to(device),
#         labels=batch["student_labels"].to(device),
#         output_hidden_states=True
#     )
# student_outputs.logits
# student_outputs.loss

In [ ]:
for epoch in range(num_epochs):
    ### 1 - Model Preparation ###
    #initializes each training epoch,
    #putting the student model in training mode
    student_model.train()
    #Initializes total_loss to track the cumulative loss for the epoch.
    total_loss = 0

    for batch_idx, batch in enumerate(dataloader):
        ### 2 - Data procesing.  ###
        #Moves the batch data to the appropriate device (CPU/GPU) for processing.
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        ### 3 - Teacher Model Inference ###
        #Disables gradient computation to save memory and speed up inference.
        with torch.no_grad():
            teacher_outputs = teacher_model(
                input_ids,
                attention_mask=attention_mask,
                output_hidden_states=True
            )
            #Applies temperature scaling to soften the teacher's predictions
            teacher_logits = teacher_outputs.logits / temperature

        ### 4 - Student Model Inference. ###
        #The student model generates logits for the same input data.
        student_outputs = student_model(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        student_logits = student_outputs.logits

        ### 5 - Compute loss ###
        # Converts logits to probabilities using softmax
        teacher_probs = F.softmax(teacher_logits, dim=-1)
        #Computes the KL Divergence between the teacher's probabilities and the student's log probabilities.
        #KL Divergence measures how well the student's predictions match the teacher's.
        student_log_probs = F.log_softmax(student_logits / temperature, dim=-1)
        loss = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean')
        #The loss is divided by accumulation_steps to balance gradient updates across accumulated batches.
        loss = loss / accumulation_steps

        ### 6- Backward pass ###
        loss.backward()

        ### 7 - Optimization Gradient Accumulation ###
        # Accumulates gradients over multiple batches
        # Updates model parameters when enough gradients are accumulated
        # Resets gradients after update
        if ((batch_idx + 1) % accumulation_steps == 0) or (batch_idx + 1 == len(dataloader)):
            optimizer.step()
            optimizer.zero_grad()

        ### 8 - Loss Tracking ###
        # Scales the loss back up by multiplying it with accumulation_steps to reflect the actual batch contribution.
        total_loss += loss.item() * accumulation_steps

        #if (batch_idx + 1) % 100 == 0:
        #    print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")

    ### 9 - Epoch-Level Reporting
    # Computes the average loss for the epoch by dividing the total loss by the number of batches.
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

Epoch 1/10, Average Loss: 26.8652
Epoch 2/10, Average Loss: 19.3893
Epoch 3/10, Average Loss: 16.8216
Epoch 4/10, Average Loss: 15.3025
Epoch 5/10, Average Loss: 14.4521
Epoch 6/10, Average Loss: 13.7981
Epoch 7/10, Average Loss: 13.3528
Epoch 8/10, Average Loss: 12.9814
Epoch 9/10, Average Loss: 12.6443
Epoch 10/10, Average Loss: 12.3276


### Store the Model

In [ ]:
distil_student_model_name = "pruned_distil_Llama-3.2-1B"

In [ ]:
student_model.save_pretrained(distil_student_model_name)
tokenizer.save_pretrained(distil_student_model_name)

In [ ]:
student_model.push_to_hub(distil_student_model_name,
                          private=False,
                          use_temp_dir=False)
tokenizer.push_to_hub(distil_student_model_name,
                      private=False,
                      use_temp_dir=False)

### Evaluating the model

In [ ]:
from lm_eval import evaluator, tasks, models

def evaluate_hf_model(model_name, tasks=['arc_easy'], num_fewshot=0):
    model_args = f"pretrained={model_name},device=cuda"
    tasks = tasks

    results = evaluator.simple_evaluate(
      model="hf",
      model_args=model_args,
      tasks=tasks,
      num_fewshot=0,
      limit=None,
      bootstrap_iters=10
    )

    metrics = results.get('results', {})
    return metrics

In [ ]:
tasks = ['lambada']

metrics_pruned_kd = evaluate_hf_model("TachyHealthResearch/pruned_distil_Llama-3.2-1B", tasks=tasks)

metrics_pruned_kd